Record Linkage - practical example

In [67]:
import pandas as pd
import recordlinkage as rl
from recordlinkage.datasets import load_febrl4
from recordlinkage.index import Block

df_a, df_b = load_febrl4()

In [84]:
#to simplify this notebook dropping some columns

df_a = df_a[['given_name', 'surname',  'date_of_birth', ]]
df_b = df_b[['given_name', 'surname',  'date_of_birth', ]]

In [70]:
df_b.shape

(5000, 4)

In [71]:
indexer = rl.Index()
indexer.full()
candidates = indexer.index(df_a, df_b)
print(len(candidates))

25000000


In [73]:
indexer = rl.Index()
indexer.sortedneighbourhood(left_on='given_name', right_on='given_name')
indexer.sortedneighbourhood(left_on='surname', right_on='surname')
candidates = indexer.index(df_a, df_b)
print(len(candidates))

190980


In [74]:
indexer = rl.Index()
indexer.block(left_on='given_name', right_on='given_name')
indexer.block(left_on='surname', right_on='surname')

candidates = indexer.index(df_a, df_b)
print(len(candidates))

159506


In [98]:
# initialise class
comp = rl.Compare()

# initialise similarity measurement algorithms
comp.string('given_name', 'given_name', method='jarowinkler',  threshold=0.85,label='given_name_match')
comp.string('surname', 'surname', method='jarowinkler', threshold=0.85 ,label='surname_match')
comp.exact('date_of_birth', 'date_of_birth', label='dob_match')

# the method .compute() returns the DataFrame with the feature vectors.
features = comp.compute(candidates, df_a, df_b)
features

given_name_match  surname_match  dob_match
rec_id_1    rec_id_2                                                  
rec-0-org   rec-0-dup-0                  1.0            1.0          1
            rec-1505-dup-0               0.0            1.0          0
            rec-1636-dup-0               0.0            1.0          0
            rec-2074-dup-0               0.0            1.0          0
            rec-2323-dup-0               1.0            0.0          0
...                                      ...            ...        ...
rec-999-org rec-675-dup-0                1.0            0.0          0
            rec-859-dup-0                0.0            1.0          0
            rec-91-dup-0                 1.0            0.0          0
            rec-911-dup-0                0.0            1.0          0
            rec-999-dup-0                1.0            1.0          1

[159506 rows x 3 columns]

In [99]:
features['score'] = features.loc[:, 'given_name_match':'dob_match'].sum(axis=1)
features.head()

given_name_match  surname_match  dob_match  score
rec_id_1  rec_id_2                                                         
rec-0-org rec-0-dup-0                  1.0            1.0          1    3.0
          rec-1505-dup-0               0.0            1.0          0    1.0
          rec-1636-dup-0               0.0            1.0          0    1.0
          rec-2074-dup-0               0.0            1.0          0    1.0
          rec-2323-dup-0               1.0            0.0          0    1.0

In [100]:
features['score'].value_counts()

1.0    154637
3.0      3158
2.0      1711
Name: score, dtype: int64

### Compare potential dupes side by side

In [79]:
#Create the index list from features
index_list = list(features.index.values)

index_a=[]
index_b=[]

for i in range(len(index_list)):
    index_a.append(index_list[i][0])
    index_b.append(index_list[i][1])

In [85]:


#Reorder the datset as per the features index and then concat them
df_a_dup=(df_a.loc[index_a]).reset_index()
df_b_dup=(df_b.loc[index_b]).reset_index()

columns = df_a_dup.columns

#subscript _1 and _2 signify which dataset
for i in columns:
    #print i
    df_a_dup[i + '_1'] = df_a_dup[i]
    df_b_dup[i + '_2'] = df_b_dup[i]
    df_a_dup.drop([i], axis=1, inplace=True)
    df_b_dup.drop([i], axis=1, inplace=True)

    
#Concat to match
df_match=pd.concat([df_a_dup, df_b_dup],
                  axis = 1)

df_match

,rec_id_1,given_name_1,surname_1,date_of_birth_1,rec_id_2,given_name_2,surname_2,date_of_birth_2
0,rec-0-org,rachael,dent,19280722,rec-0-dup-0,rachael,dent,19280722
1,rec-0-org,rachael,dent,19280722,rec-1505-dup-0,emiily,dent,19960112
2,rec-0-org,rachael,dent,19280722,rec-1636-dup-0,layla,dent,19730408
3,rec-0-org,rachael,dent,19280722,rec-2074-dup-0,mitchell,dent,19900531
4,rec-0-org,rachael,dent,19280722,rec-2323-dup-0,rachael,yallop,19760214
...,...,...,...,...,...,...,...,...
159501,rec-999-org,jacob,lowe,19761225,rec-675-dup-0,jacob,margrie,19130517
159502,rec-999-org,jacob,lowe,19761225,rec-859-dup-0,macey,lowe,19081128
159503,rec-999-org,jacob,lowe,19761225,rec-91-dup-0,jacob,clarke,19181001
159504,rec-999-org,jacob,lowe,19761225,rec-911-dup-0,jack,lowe,19091212


In [107]:
#Concat feature to match
df_match=pd.concat([df_match, features.reset_index()[features.columns]], axis = 1)

df_match[df_match.score>1]

,rec_id_1,given_name_1,surname_1,date_of_birth_1,rec_id_2,given_name_2,surname_2,date_of_birth_2,given_name_match,surname_match,dob_match,score
0,rec-0-org,rachael,dent,19280722,rec-0-dup-0,rachael,dent,19280722,1.0,1.0,1,3.0
10,rec-1-org,isabella,everett,19110816,rec-1-dup-0,isabella,everett,19110816,1.0,1.0,1,3.0
47,rec-10-org,lachlan,reid,19500531,rec-10-dup-0,lachlnn,reid,19500531,1.0,1.0,1,3.0
125,rec-10-org,lachlan,reid,19500531,rec-4572-dup-0,lachlan,red,19600906,1.0,1.0,0,2.0
146,rec-100-org,hayden,stapley,NaN,rec-100-dup-0,hayden,stapley,NaN,1.0,1.0,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
159297,rec-996-org,emiily,finlay,19070731,rec-1692-dup-0,emiily,finalay,19800129,1.0,1.0,0,2.0
159360,rec-996-org,emiily,finlay,19070731,rec-996-dup-0,emiily,finlay,19070731,1.0,1.0,1,3.0
159374,rec-997-org,madeline,whithear,19090324,rec-997-dup-0,madeline,whithear,19090324,1.0,1.0,1,3.0
159455,rec-998-org,bailey,green,19960416,rec-998-dup-0,bailey,gree n,19960416,1.0,1.0,1,3.0
